In [1]:
!pip install sklearn
!pip install scikit_learn
!pip install pandas

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2343 sha256=1d434a1b9d5155d7459bb5f4a6799e68bf7aa8ed9cd37977a9c2d120ef67ee0f
  Stored in directory: /root/.cache/pip/wheels/b7/34/ce/c0558fa3c14b4485fa383d6fba29a76c9f41531f818a86e4c2
Successfully built sklearn
Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 9.8 MB 17.8 MB/s eta 0:00:01
     |████████████████████████████████| 9.7 MB 56.5 MB/s eta 0:00:01
     |████████████████████████████████| 31.2 MB 76.5 MB/s eta 0:00:01
     |████████████████████████████████| 306 kB 30.8 MB/s eta 0:00:01
     |████████████████████████████████| 33.8 MB 22.4 MB/s eta 0:00:01     |█████████▉                      | 10.4 MB 31.0 MB/s eta 0:00:01
Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 12.2 MB 30.7 MB/s eta 0:00:01


In [3]:
!pip install tensorflow

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 588.3 MB 46 kB/s s eta 0:00:01     |█████████▌                      | 175.5 MB 22.7 MB/s eta 0:00:19     |██████████▋                     | 194.3 MB 25.9 MB/s eta 0:00:16     |█████████████                   | 237.9 MB 20.0 MB/s eta 0:00:18     |███████████████                 | 277.2 MB 22.3 MB/s eta 0:00:14     |███████████████▎                | 281.4 MB 22.3 MB/s eta 0:00:14     |███████████████▌                | 285.7 MB 24.8 MB/s eta 0:00:13     |██████████████████▏             | 334.1 MB 22.3 MB/s eta 0:00:12     |███████████████████████▊        | 436.7 MB 20.3 MB/s eta 0:00:08     |█████████████████████████▉      | 474.5 MB 19.4 MB/s eta 0:00:06     |██████████████████████████████▌ | 560.1 MB 21.7 MB/s eta 0:00:02
     |████████████████████████████████| 439 kB 30.0 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 29.3 MB/s eta 0:00:01
     |█████████████

In [3]:
import numpy as np
import keras as K
import  tensorflow as  tf
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.datasets import cifar10


# 加载数据
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# 处理数据
num_pixels = X_train.shape[1] * X_train.shape[2] * X_train.shape[3]
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

# 标准化数据
X_train /= 255
X_test /= 255

# 对标签进行one-hot编码
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

# 定义模型
model = Sequential()
model.add(Dense(512, input_dim=num_pixels, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# 编译模型
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 训练模型
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

# 计算每个神经元互信息
from sklearn.feature_selection import mutual_info_classif
import pandas as pd

# 获取中间层的输出
get_layer_output = K.backend.function([model.layers[0].input],
                                  [model.layers[1].output])
layer_output = get_layer_output([X_train])[0]

# 计算互信息
mi = mutual_info_classif(layer_output, y_train[:,0])
mi_df = pd.DataFrame({'mi': mi})

# 对互信息较低的神经元进行剪枝
low_mi_indices = mi_df[mi_df['mi'] < 0.05].index
pruned_model = Sequential()
pruned_model.add(Dense(512, input_dim=num_pixels, activation='relu'))

# 删除互信息较低的神经元对应的列，并随机初始化一些新的列
old_weights = model.layers[1].get_weights()[0]
old_biases = model.layers[1].get_weights()[1]
new_weights = np.delete(old_weights, low_mi_indices, axis=1)
num_new_cols = old_weights.shape[1] - new_weights.shape[1]
new_cols = np.random.randn(new_weights.shape[0], num_new_cols) * 0.01
new_weights = np.concatenate([new_weights, new_cols], axis=1)

pruned_model.add(Dense(new_weights.shape[1], activation='relu'))
pruned_model.add(Dense(num_classes, activation='softmax'))

# 拷贝训练好的参数
for i, layer in enumerate(pruned_model.layers):
    if i == 0:
        layer.set_weights(model.layers[i].get_weights())
    elif i == 1:
        layer.set_weights([new_weights, old_biases])
    else:
        layer.set_weights(model.layers[i].get_weights())

# 重新编译模型
pruned_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 重新训练
pruned_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

2023-03-22 17:34:42.375891: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 17:34:43.026679: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78977 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:6b:01.0, compute capability: 8.0


Epoch 1/10


2023-03-22 17:34:46.186614: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-22 17:34:46.344457: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f39e729f6a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-22 17:34:46.344572: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2023-03-22 17:34:46.349341: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-22 17:34:46.499342: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


250/250 [==============================] - 3s 4ms/step - loss: 1.9408 - accuracy: 0.3157 - val_loss: 1.7889 - val_accuracy: 0.3547
Epoch 2/10
250/250 [==============================] - 1s 5ms/step - loss: 1.7068 - accuracy: 0.3911 - val_loss: 1.6759 - val_accuracy: 0.4031
Epoch 3/10
250/250 [==============================] - 1s 3ms/step - loss: 1.6242 - accuracy: 0.4212 - val_loss: 1.5980 - val_accuracy: 0.4279
Epoch 4/10
250/250 [==============================] - 1s 5ms/step - loss: 1.5552 - accuracy: 0.4474 - val_loss: 1.5590 - val_accuracy: 0.4501
Epoch 5/10
250/250 [==============================] - 1s 3ms/step - loss: 1.5225 - accuracy: 0.4600 - val_loss: 1.5219 - val_accuracy: 0.4503
Epoch 6/10
250/250 [==============================] - 1s 4ms/step - loss: 1.4668 - accuracy: 0.4804 - val_loss: 1.5308 - val_accuracy: 0.4591
Epoch 7/10
250/250 [==============================] - 1s 4ms/step - loss: 1.4437 - accuracy: 0.4857 - val_loss: 1.4701 - val_accuracy: 0.4796
Epoch 8/10
250/25